In [17]:
import rospy
import actionlib
import actionlib.msg
import assignment_2_2023.msg
from std_srvs.srv import *
import sys
import select
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Twist
from assignment_2_2023.msg import Vel
from assignment_2_2023.msg import PlanningAction, PlanningGoal, PlanningResult, PlanningActionResult
from actionlib_msgs.msg import GoalStatus
from sensor_msgs.msg import LaserScan
import math
import ipywidgets as widgets
from IPython.display import display 
from ipywidgets import Layout, HBox
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import interact, interactive, fixed, interact_manual
import tf
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
import numpy as np
import matplotlib as mpl
from assignment_2_2023.srv import Last_target

In [18]:
# Define state descriptions
state_desc_ = {
    0: 'Go to point',
    1: 'Wall following',
    2: 'Done',
    6: 'Goal cancelled'
}

In [19]:
# Publisher node function to handle position values and publish them
def publisher_node(msg):
    global pub
    
    # Get the actual position and velocity
    actual_pos = msg.pose.pose.position
    actual_vel_linear = msg.twist.twist.linear
    actual_vel_angular = msg.twist.twist.angular
    
    # Update position and velocity display
    position_display.value = f"<b>Position:</b> X: {actual_pos.x}, Y: {actual_pos.y}<br><b>Velocity:</b> Linear: {actual_vel_linear.x}, Angular: {actual_vel_angular.z}"
    
    # Create a Vel message
    my_pos_and_vel = Vel()
    my_pos_and_vel.pos_x = actual_pos.x
    my_pos_and_vel.pos_y = actual_pos.y
    my_pos_and_vel.vel_x = actual_vel_linear.x
    my_pos_and_vel.vel_z = actual_vel_angular.z
    
    # Publish the position and velocity
    pub.publish(my_pos_and_vel)

In [20]:
# Initialize publisher
pub = rospy.Publisher('/pos_vel', Vel, queue_size=10)

In [21]:
def set_goal(input_x, input_y):
    goal = assignment_2_2023.msg.PlanningGoal()
    
    # Update ros parameters
    rospy.set_param('/des_pos_x', input_x)
    rospy.set_param('/des_pos_y', input_y)

    # Set goal parameters
    goal.target_pose.pose.position.x = rospy.get_param('/des_pos_x')
    goal.target_pose.pose.position.y = rospy.get_param('/des_pos_y')
    #goal.target_pose.pose.position.x = input_x
    #goal.target_pose.pose.position.y = input_y

    # send goal to the service
    client.send_goal(goal)
    rospy.loginfo("Inserted goal: des_x = %f, des_y = %f", input_x, input_y)

In [22]:
def cancel_goal():
    if client.get_state() == actionlib.GoalStatus.ACTIVE:
        client.cancel_goal()
        rospy.loginfo("Goal cancelled")
    else:
        rospy.loginfo("No active goal to cancel")

In [23]:
# Update status_update function
def status_update(msg):
    global current_status
    if msg.status_list:
        current_status = msg.status_list[0].status
        # print("Received current status:", current_status)
        if current_status in state_desc_:
            status_text.value = f"Current Status: {state_desc_[current_status]} ({current_status})"
        else:
            status_text.value = f"Current Status: Unknown ({current_status})"
    else:
        status_text.value = "Current Status: Unknown"

In [24]:
# Check if ROS node has already been initialized
if not rospy.core.is_initialized():
    rospy.init_node('set_target_client')

In [25]:
client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
client.wait_for_server()

True

In [26]:
# Create widgets for setting goal
goal_x = widgets.FloatSlider(min=-15.0, max=15.0, step=0.1, description='desired x:')
goal_y = widgets.FloatSlider(min=-15.0, max=15.0, step=0.1, description='desired y:')
set_goal_button = widgets.Button(description="Set Goal")
cancel_goal_button = widgets.Button(description="Cancel Goal")

In [27]:
# Create widgets for status update
status_text = widgets.Label(value="Current Status: Unknown")
# Create widget for displaying position and velocity
position_display = widgets.HTML(value="<b>Position:</b> X: ?, Y: ?<br><b>Velocity:</b> Linear: ?, Angular: ?")

In [28]:
# Define actions for button clicks
def set_goal_click(b):
    set_goal(goal_x.value, goal_y.value)

def cancel_goal_click(b):
    cancel_goal()

In [29]:
# Assign actions to buttons
set_goal_button.on_click(set_goal_click)
cancel_goal_button.on_click(cancel_goal_click)

In [30]:
# Subscribe to goal status updates
goal_status_sub = rospy.Subscriber('/reaching_goal/status', GoalStatus, status_update)

In [31]:
# Style widgets 
goal_x.layout.width = '400px'
goal_y.layout.width = '400px'
set_goal_button.style.button_color = 'lightblue'
cancel_goal_button.style.button_color = 'lightcoral'
status_text.layout.margin = '20px 0'
position_display.layout.margin = '20px 0'

layout = widgets.Layout(display='flex', flex_flow='column', align_items='center')

# Display widgets
display(widgets.VBox([goal_x, goal_y, set_goal_button, cancel_goal_button, status_text, position_display], layout=layout))

[INFO] [1712154293.709163, 1476.243000]: Inserted goal: des_x = 2.000000, des_y = -2.500000
[INFO] [1712154296.653864, 1478.408000]: Goal cancelled
[INFO] [1712154297.448239, 1478.977000]: Inserted goal: des_x = 2.000000, des_y = -2.500000
[INFO] [1712154299.352065, 1480.432000]: Goal cancelled
[INFO] [1712154301.385125, 1482.108000]: No active goal to cancel
